In [2]:
#TensorFlow model that predicts closing price using factors such as previous adjusted closing values, various value ratios, volatility, etc.
import tensorflow as tf
import pandas as pd
import numpy as np
df = pd.read_csv("SNPData.csv", low_memory=False)
#cleaning up dataframe.
df.drop(df.columns[[1,2,3]], axis=1, inplace=True)
df.drop([1, 0], inplace=True)
df.columns = df.columns.str.replace('.1', '.Open')
df.columns = df.columns.str.replace('.2', '.Volume')
df.columns = df.columns.str.replace('Ticker', 'Date')
!pip install ludwig[full]
df

<ipython-input-2-92f10f82b26f>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('.1', '.Open')
<ipython-input-2-92f10f82b26f>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('.2', '.Volume')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   

,Date,A,A.Open,A.Volume,AAL,AAL.Open,AAL.Volume,AAPL,AAPL.Open,AAPL.Volume,...,TECH.Volume,WRB,WRB.Open,WRB.Volume,XOM,XOM.Open,XOM.Volume,XRAY,XRAY.Open,XRAY.Volume
2,2020-09-09,96.09,94.3,954400,13.05,13.48,79030200,115.24,115.18,176940500,...,611600,40.12,39.76,1096800,32.77,33.11,25886100,42.79,42.48,994700
3,2020-09-10,96.72,97.03,1933200,13.01,13.21,52318600,111.48,118.22,182274400,...,403200,39.39,40.17,900000,31.95,33.04,34381200,42.3,42.9,927700
4,2020-09-11,96.75,96.86,1368600,12.95,13.08,43916700,110.01,112.54,180860300,...,391600,39.71,39.54,668850,31.86,32.06,26838600,42.73,42.55,1287000
5,2020-09-14,97.56,97.86,1207700,13.26,13.02,39859100,113.31,112.68,140150100,...,425200,40.4,39.93,947550,31.65,31.87,26129900,44.41,43.08,1176200
6,2020-09-15,98.49,97.85,1053800,13.14,13.27,38958800,113.49,116.23,184642000,...,558800,40.07,40.59,683550,31.32,31.77,23724200,44.85,44.51,1213400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,2023-09-01,121.91,122.69,840700,14.68,14.8,14713500,189.46,189.49,45732600,...,511000,62.43,62.31,552300,113.52,112.2,14828400,37.48,37.41,1784900
753,2023-09-05,118.73,121.64,1270600,14.3,14.53,25690400,189.7,188.28,45280000,...,601700,61.75,62.42,720600,113.53,114.1,18304000,36.69,37.35,1592100
754,2023-09-06,117.94,118.35,1668900,14.33,14.27,25490000,182.91,188.4,81755800,...,766000,61.53,61.54,660700,114.51,113.48,13325500,36.7,36.62,991600
755,2023-09-07,116.26,117.97,1317000,14.0,14.25,24678300,177.56,175.18,112488800,...,762700,61.78,61.76,1309100,113.95,114.65,13248500,36.16,36.63,843000


In [3]:
#adding the CBOE Volatility Index (VIX) to dataframe
VIX = pd.read_csv("^VIX-2.csv", low_memory=False)
df['Market Volatility'] = VIX['Adj Close']
df['Market Volatility'].fillna(value = df['Market Volatility'].mean(), inplace = True)
df

,Date,A,A.Open,A.Volume,AAL,AAL.Open,AAL.Volume,AAPL,AAPL.Open,AAPL.Volume,...,WRB,WRB.Open,WRB.Volume,XOM,XOM.Open,XOM.Volume,XRAY,XRAY.Open,XRAY.Volume,Market Volatility
2,2020-09-09,96.09,94.3,954400,13.05,13.48,79030200,115.24,115.18,176940500,...,40.12,39.76,1096800,32.77,33.11,25886100,42.79,42.48,994700,26.870001
3,2020-09-10,96.72,97.03,1933200,13.01,13.21,52318600,111.48,118.22,182274400,...,39.39,40.17,900000,31.95,33.04,34381200,42.3,42.9,927700,25.850000
4,2020-09-11,96.75,96.86,1368600,12.95,13.08,43916700,110.01,112.54,180860300,...,39.71,39.54,668850,31.86,32.06,26838600,42.73,42.55,1287000,25.590000
5,2020-09-14,97.56,97.86,1207700,13.26,13.02,39859100,113.31,112.68,140150100,...,40.4,39.93,947550,31.65,31.87,26129900,44.41,43.08,1176200,26.040001
6,2020-09-15,98.49,97.85,1053800,13.14,13.27,38958800,113.49,116.23,184642000,...,40.07,40.59,683550,31.32,31.77,23724200,44.85,44.51,1213400,26.459999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,2023-09-01,121.91,122.69,840700,14.68,14.8,14713500,189.46,189.49,45732600,...,62.43,62.31,552300,113.52,112.2,14828400,37.48,37.41,1784900,13.330000
753,2023-09-05,118.73,121.64,1270600,14.3,14.53,25690400,189.7,188.28,45280000,...,61.75,62.42,720600,113.53,114.1,18304000,36.69,37.35,1592100,13.630000
754,2023-09-06,117.94,118.35,1668900,14.33,14.27,25490000,182.91,188.4,81755800,...,61.53,61.54,660700,114.51,113.48,13325500,36.7,36.62,991600,13.930000
755,2023-09-07,116.26,117.97,1317000,14.0,14.25,24678300,177.56,175.18,112488800,...,61.78,61.76,1309100,113.95,114.65,13248500,36.16,36.63,843000,16.090000


In [128]:
#obtains user input for stock of interest and creates filtered dataframe
stock_interest = None;
while stock_interest not in df.columns:
  stock_interest = input('Please enter your desired stock\'s ticker: ')
filtered_data = df.filter(['Date',stock_interest, stock_interest + '.Open', stock_interest + '.Volume', 'Market Volatility'], axis=1)
filtered_data["Tomorrow's price"] = filtered_data[stock_interest].shift(-1)
filtered_data['Increases Tomorrow'] = (filtered_data["Tomorrow's price"] > filtered_data[stock_interest]).astype(int); # will the closing price of tomorrow be higher than today's closing price?
filtered_data = filtered_data.drop(columns=['Date'])
filtered_data

Please enter your desired stock's ticker: AAPL


,AAPL,AAPL.Open,AAPL.Volume,Market Volatility,Tomorrow's price,Increases Tomorrow
2,115.24,115.18,176940500,26.870001,111.48,0
3,111.48,118.22,182274400,25.850000,110.01,0
4,110.01,112.54,180860300,25.590000,113.31,1
5,113.31,112.68,140150100,26.040001,113.49,1
6,113.49,116.23,184642000,26.459999,110.14,0
...,...,...,...,...,...,...
752,189.46,189.49,45732600,13.330000,189.7,1
753,189.7,188.28,45280000,13.630000,182.91,0
754,182.91,188.4,81755800,13.930000,177.56,0
755,177.56,175.18,112488800,16.090000,178.18,1


In [131]:
# config = {
#     "input_features": [
#         {
#             "name": stock_interest,
#             "type": "number"
#         },
#         {
#             "name": stock_interest + '.Open',
#             "type": "number"
#         },
#         {
#             "name": stock_interest + '.Volume',
#             "type": "number"
#         },
#         {
#             "name": "Market Volatility",
#             "type": "number"
#         }
#     ],
#     "output_features": [
#         {
#             "name": "Increases Tomorrow",
#             "type": "binary"
#         }
#     ]
# }
# #designing the model
input_obj = tf.keras.Input(shape=(4,))  # four dimensional input vector
first_hidden = tf.keras.layers.Dense(8, activation='relu')(input_obj)
second_hidden = tf.keras.layers.Dense(4, activation='relu')(first_hidden)
output_layer = tf.keras.layers.Dense(1,activation = 'sigmoid')(second_hidden)  # Output layer

model = tf.keras.Model(inputs = input_obj, outputs = output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Define the model
# model = tf.keras.Sequential()

# # Input layer
# model.add(tf.keras.layers.Dense(32, activation='relu', input_shape=(4,)))

# # Hidden layers
# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))  # Adding dropout for regularization
# model.add(tf.keras.layers.Dense(16, activation='relu'))

# # Output layer
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Print a summary of the model's architecture
# model.summary()


In [132]:
 # splitting data into train and validation
# from ludwig.api import LudwigModel
# from ludwig.hyperopt.run import hyperopt

# model = LudwigModel(config)

# train_stats, _, model_dir = model.train(training_set)
# predictions = model.predict(training_set)
# reality = training_set['Increases Tomorrow'].values
# model_predictions = predictions[0]['Increases Tomorrow_predictions'].values
# count = 0
# for i in range(len(reality)):
#   if reality[i] == model_predictions[i]:
#     count+= 1
# print('accuracy: ', count/len(model_predictions))
# print(predictions.byvalue[0] == filtered_data['Increases Tomorrow'][0])
# count = 0
# for i in range(len(predictions)):
#   if predictions[i] == training_set['Increases Tomorrow'].astype('int32')[i]:
#     count+=1
# print(count)
validation_start_index = int(0.8 * len(filtered_data))
training_set = filtered_data.iloc[:validation_start_index] # splits 80% of the data for training
train_outputs = np.array(training_set['Increases Tomorrow'])
train_inputs = np.array(training_set.iloc[:, 1:5].astype('float32'))# selects input columns from data frame

testing_set = filtered_data.iloc[validation_start_index:]
testing_outputs = np.array(testing_set['Increases Tomorrow'])
testing_inputs = np.array(testing_set.iloc[:, 1:5].astype('float32'))
model.fit(train_inputs, train_outputs, batch_size = 32, epochs = 10, validation_data = (testing_inputs, testing_outputs))
for x in train_inputs:
  print(model.predict(x.reshape(1, -1)),)

Epoch 1/10
19/19 [==============================] - 1s 19ms/step - loss: 5896573.0000 - accuracy: 0.5099 - val_loss: nan - val_accuracy: 0.5430
Epoch 2/10
19/19 [==============================] - 0s 7ms/step - loss: 2572074.5000 - accuracy: 0.5099 - val_loss: nan - val_accuracy: 0.5430
Epoch 3/10
19/19 [==============================] - 0s 6ms/step - loss: 363333.4688 - accuracy: 0.5166 - val_loss: nan - val_accuracy: 0.5430
Epoch 4/10
19/19 [==============================] - 0s 6ms/step - loss: 68729.0234 - accuracy: 0.5331 - val_loss: nan - val_accuracy: 0.4636
Epoch 5/10
19/19 [==============================] - 0s 7ms/step - loss: 9540.3789 - accuracy: 0.5232 - val_loss: nan - val_accuracy: 0.4636
Epoch 6/10
19/19 [==============================] - 0s 6ms/step - loss: 30852.4805 - accuracy: 0.5530 - val_loss: nan - val_accuracy: 0.4636
Epoch 7/10
19/19 [==============================] - 0s 6ms/step - loss: 32814.6641 - accuracy: 0.4901 - val_loss: nan - val_accuracy: 0.5430
Epoch 8/

KeyboardInterrupt: ignored